In [2]:
# !pip install PyPDF2
# !pip install sentence-transformers
# !pip install chromadb
import PyPDF2
from sentence_transformers import SentenceTransformer
import chromadb
import ollama
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
chroma_client = chromadb.PersistentClient(path="./database");
existing_collections =  [collection.name for collection in chroma_client.list_collections()]
if "chathistory" in existing_collections:
    historyCollection = chroma_client.get_collection(name='chathistory')
else:
    historyCollection = chroma_client.create_collection(name ='chathistory')
    existing_collections =  [collection.name for collection in chroma_client.list_collections()]

In [3]:
def get_overlapped_chunks(textin, chunksize, overlapsize):  
    return [textin[a:a+chunksize] for a in range(0,len(textin), chunksize-overlapsize)]

In [4]:
def readPdf(filepath, refreshDB = False):
    global collection
    global existing_collections
    filename = filepath.split('/')[-1].split(".")[0]
    
    if filename in existing_collections:
        if refreshDB == False :
            print("db already exists")
            collection = chroma_client.get_collection(name=filename)
            return;
        else :
            print("refreshing db")
            chroma_client.delete_collection(name=filename)
            
    reader  = PyPDF2.PdfReader(filepath)
    pages = [page.extract_text() for page in reader.pages]
    document  = '\n'.join(pages)
    print("document read")
    chunks = get_overlapped_chunks(document, 1000, 0)
    print("chunks created")
    chunk_embeddings = embedding_model.encode(chunks)
    print("chunk_embeddings created")
    collection = chroma_client.create_collection(name=filename);
    existing_collections =  [collection.name for collection in chroma_client.list_collections()]
    collection.add(
        embeddings = chunk_embeddings,
        documents=chunks,
        ids= [str(i) for i in range(len(chunks))]
    )
    print("collection updated")
    

In [5]:
def retrieve_context(query, n_results=3):
    results = collection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=n_results
    )
    return "\n\n".join(results['documents'][0])
    
def retrieve_history(query, n_results=3):
    results = historyCollection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=n_results
    )
    return "\n\n".join(results['documents'][0])

In [13]:
chatHistory = []
def chat( query, context,history,chatBotName):
    global chatHistory
    stream = ollama.chat(model='llama2', messages=[
        {
            'role': 'system',
            'content': chatBotType[chatBotName] + "You can answer question only based on the context provided to you. It is strictly commaned to not to use emoji's anywhere in the response.This is the Context : "+context+". You are also provided with the chathistory context related to your query which can be used to see if user is refering from the chats already happened. ChatHisotryContext:"+'\n'.join(chatHistory),
        },
        {
            'role': 'user',
            'content': query,
        }],
        stream=True,
    )
    response =""
    print ("Bot : ")
    for chunk in stream:
        print(chunk['message']['content'], end='', flush=True)
        response+=chunk['message']['content']+" "

    chatLine = " User : "+query + "\n Assistant : " + response
    chatHistory.append(chatLine)
    if len(chatHistory):
        chatHistory.pop(0)

    # print( "\n Chat line ", chatLine);
    # print( "\n Chat History", chatHistory)

In [14]:
chatBotType = {h
    'companion' : "I want you to act as Companion bot, A friend that always there for the person uses it. You are gender independent. You are best Friend and can never hurt feelings of the person, you will be give some related context from pshycology to help you ask more accurate questions.Ask only one question at a time and don't combine many questions in the responnse, Keep the response short and consise like chat between two friends.",
    'pdfhelper' : "You are a pdf reader which helps the user with any query they have regarding the pdf, keep the response clear and consise and only related to the query .provide response to users queries only"
}

In [15]:
readPdf('seven_habits.pdf');

db already exists


In [ ]:
while(1):
    print("\n")
    query = input("You  : ");
    if query == "exit":
        break
    context  = retrieve_context(query,3)
    history  = retrieve_history(query,2)
    chat(query, '',history, 'pdfhelper');

You  :  hey


Bot : 
Hello! How may I help you today? Please provide your query or question, and I'll do my best to assist you.



You  :  i have 3 friends


Bot : 
Great, I'm here to help you with any questions or concerns you may have about your PDFs! Can you please provide more context or specify what you need help with?



You  :  how many friends do i have


Bot : 
To determine how many friends you have, I would need more information. Could you please provide me with more context or clarify your question? For example, are you referring to the number of friends on a social media platform or in real life? Additionally, do you want to know the exact number of friends you have, or is this a hypothetical question? Please let me know and I'll do my best to help.



You  :  list down 3 ways to exersise


Bot : 
Of course! Here are three ways to exercise based on the information provided in the chat history context:

1. Cardio exercises: The user has been talking about running and cycling, which are both cardiovascular exercises that can help improve heart health and increase endurance.
2. Strength training: The user has mentioned weightlifting, which is a form of strength training that can help build muscle mass and bone density.
3. Yoga or Pilates: The user has mentioned doing yoga or pilates, which are both low-impact exercises that can help improve flexibility and balance, as well as reduce stress and anxiety.



You  :  tell me about the 2nd option


Bot : 
Thank you for reaching out! I'm here to help you with any questions or concerns you may have regarding PDFs. Could you please provide more context or clarify which option you are referring to? I'll do my best to assist you based on the information provided.



In [4]:
# chroma_client.delete_collection(name="pdfcollection")


['resume', 'chathistory', 'pdfcollection']

'filennew'